# NCSU GIS 582: Geospatial Modeling and Analysis

## Introduction to Jupyter Notebooks Tutorial B

### Getting Started with GRASS

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

This notebook is based on sections of [GIS-based Analysis of Coastal Lidar Time-Series by Hardin et al (2014)](https://link.springer.com/book/10.1007/978-1-4939-1835-5).

In this notebook we will:
* [Import Python and GRASS Python API packages](#1.-Import-Python-Packages)
* [Create a new GRASS project](#2.-Create-a-New-Project)
* [Import data](#4.-Import-Data)
* Get a quick overview of [the Python API](#5.-GRASS-Python-API)
* [Visualize data](#6.-Data-Visualization-with-grass.jupyter)

## Google Colab Setup

Let’s first print system description to know where are we.

In [ ]:
!lsb_release -a

At the time of writing this tutorial, Colab has Linux Ubuntu 22.04.4 LTS. So we add the ppa:ubuntugis repository, update and install GRASS. It might take a couple of minutes according to the resources available.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

***

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
import os
from pathlib import Path

We are going to import the GRASS Python API (`grass.script`) and the GRASS Jupyter package (`grass.jupyter`), but first, we need to find the path to those packages using the `--config python_path` command. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS packages we need.
import grass.script as gs
import grass.jupyter as gj

***

## 2. Create a New Project

In [ ]:
gs.create_project("nags_head", epsg=3358, overwrite=True)

***

## 3. Start GRASS Session

In [ ]:
home = os.environ['HOME']

In [ ]:
gj.init(home+"/GIS582-assignments/nags_head/PERMANENT");

We've launched GRASS now! We can access GRASS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

In [ ]:
!g.region -p

***

## 4. Import Data

For this tutorial, we will import a NAIP image of Nags Head.

In [ ]:
!r.import input="./naip_2020.tif" output="naip_2020" resolution=value resolution_value=1

Look at the computational region above. Since it's never been set before, it defaults to an area that's 1m by 1m! Let's change the computation region to match the imagery we just imported. How many cells are in the computation area now?

In [ ]:
!g.region raster="naip_2020.1" -p

***

## 5. GRASS Python API

There are multiple Python APIs for accessing GRASS tools' functionality - [GRASS Python Scripting Library](https://grass.osgeo.org/grass85/manuals/python_intro.html). Here, we will be using `grass.script` because `grass.tools` is in the preview version of GRASS but not the stable release that we are using. 

The `grass.script` provides functions to call GRASS tools within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file

Here's an example of the Python API in action:

In [ ]:
gs.run_command("g.list", type="raster")

**Try it yourself!**

_The `r.info map=NAME` command will print information about the raster NAME. Execute `r.info` in Python to get information about `'naip_2020.1'`._

<details>
    <summary>👉 <b>click to see an example</b></summary>
    
```python
gs.run_command("r.info", map="naip_2020.1")
```
</details>

The Python API also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`

**Try it yourself!**

_Try using `gs.raster_info` to print information about NAME._

<details>
    <summary>👉 <b>click to see an example</b></summary>
    
```python
gs.raster_info('naip_2020.1')
```
</details>

***

## 6. Data Visualization with `grass.jupyter`

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "\_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="naip_2020.1", flags="i") # d.rast map=naip_2020.1
example.d_barscale(bgcolor="none") # d.barscale
example.show()

To display the image, we call the `show()` method. You can also save the image with the `save()` method.

We also might want to make this a nice square. Instead of clipping the image, we can adjust the computational region.

Here we use the `grow` parameter with a negative value to shrink the region by 100 m on each side. See the [`g.region`](https://grass.osgeo.org/grass-stable/manuals/g.region.html) manual page for more options.

In [ ]:
!g.region grow=-100 -p

Now, it's a nice rectangle!

In [ ]:
example = gj.Map(use_region=True)
example.d_rast(map="naip_2020.1") # d.rast map=naip_2020.1
example.d_barscale() # d.barscale
example.show()

Let's save the region so we can use it later.

In [ ]:
!g.region --help

In [ ]:
!g.region save=jockeys_ridge

And now we're off and running with GRASS in Jupyter Notebooks!

In this tutorial, we learned:
1. How to launch a GRASS project in a Jupyter Notebook (including importing data and managing computational region)
2. How to execute GRASS commands using the command line interface and with the Python API
3. How to create basic visualizations of GRASS data

## Acknowledgements

These materials were created with support from the National Science Foundation (Award [2303651](https://www.nsf.gov/awardsearch/showAward?AWD_ID=2303651)), the Digital Education and Learning Technologies Applications (DELTA) Center at NC State University and the [Center for Geospatial Analytics](https://cnr.ncsu.edu/geospatial/) at NC State.